# Wrangling and cleaning data for geospatial plotting for New York City Bike 2022 

## Loading Libraries and data set

In [7]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt

In [8]:
import gc # this is garbage collector - speeds up performance
gc.collect()

9541

In [9]:
from dotenv import load_dotenv 
import os  
load_dotenv() 

True

In [ ]:
# Get the full file path from .env
Citybike_path = os.getenv("CITIBIKE_2022_WEATHER")

# Read the CSV directly
stations_df = pd.read_csv(
    Citybike_path,
    usecols=[
        'start_station_name',
        'end_station_name',
        'duration',
        'start_lat',
        'start_lng',
        'end_lat',
        'end_lng',
        'value'
    ]
)

In [27]:
stations_df.head(10)

,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,value
0,S 4 St & Wythe Ave,Kent Ave & S 11 St,40.712859,-73.965903,40.707645,-73.968415,1
1,Lexington Ave & E 120 St,Amsterdam Ave & W 125 St,40.801307,-73.939817,40.813358,-73.956461,1
2,Lexington Ave & E 120 St,Amsterdam Ave & W 125 St,40.801307,-73.939817,40.813358,-73.956461,1
3,Cleveland Pl & Spring St,Mott St & Prince St,40.722104,-73.997249,40.723180,-73.994800,1
4,Central Park North & Adam Clayton Powell Blvd,E 110 St & Madison Ave,40.799484,-73.955613,40.796154,-73.947821,1
5,Central Park North & Adam Clayton Powell Blvd,E 110 St & Madison Ave,40.799484,-73.955613,40.796154,-73.947821,1
6,E 2 St & 2 Ave,Allen St & Rivington St,40.725029,-73.990697,40.720196,-73.989978,1
7,Broadway & E 14 St,E 25 St & 1 Ave,40.734546,-73.990741,40.738177,-73.977387,1
8,Cathedral Pkwy & Broadway,E 110 St & Madison Ave,40.804213,-73.966991,40.796154,-73.947821,1
9,Grand St & Elizabeth St,Mott St & Prince St,40.718822,-73.995960,40.723180,-73.994800,1


In [6]:
stations_df.shape

(29838166, 7)

In [7]:
stations_df.dtypes

start_station_name     object
end_station_name       object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
value                   int64
dtype: object

In [8]:
df_nan= stations_df.isna().sum()
df_nan

start_station_name       49
end_station_name      69884
start_lat                 0
start_lng                 0
end_lat               37223
end_lng               37223
value                     0
dtype: int64

## Data cleaning Plan
### Problem: The data are devided into 4 groups:

Group 1: Raws with starting, ending station names and GPS coordinates are missing.i.e complete missing information to answer the geospatial questions. those raws must be cleaned out.

Group 2: missing satrting station name but available end station name. those wil be imputed with "missing start" and expected to be less than 49.

Group 3: missing end station names but available coordinates. those can be restored from the coordinates.

Group 4: Available starting station but missing end station name and coordinates. those should be imputed with "missing end". Goal: Keep these in the dataset so your total trip counts remain accurate, even if their destination is unknown.

### Cleaning Steps:

#### Step 1: The Purge — Drop rows where everything is NaN
Remove rows that have no usable information at all (e.g., missing both station names and GPS). This keeps the dataset clean from completely empty entries.

#### Step 2: Tagging Irrecoverable Data
Identify rows where the end station has no name AND incomplete GPS.
These rows cannot be recovered later, so they are tagged with a clear label (“Missing GPS and Name”) instead of being dropped.
This preserves them for counting but marks them as unusable for recovery.
### Recovering missing names through the coordinates information
#### Step 3: Build a Recovery Table (GPS → Station Name)
Create a compact reference table that contains only valid end station rows, each with a unique pair of coordinates and its correct station name.
This table acts as a clean lookup source for restoring missing names later.

#### Step 4: Attach Recovered Names via Merge
Merge the recovery table back into the main dataset using the end‑station coordinates.
This adds a temporary column containing the correct station name wherever a match exists.
Because merging is vectorized, it handles millions of rows efficiently.

#### Step 5: Finalize the Restoration
Fill in the missing end station names using the recovered values from the merge.
After the restoration is complete, remove the temporary helper column to return the dataset to its original structure.

#### Final cleanup: Any remaining NaNs get a general "Unknown" label
citibike_weather_df['end_station_name'] = citibike_weather_df['end_station_name'].fillna("Unknown Station")

Note: Station id cannot be taken as name alternative because every station name might have several id.


## Cleaning and Tagging missing values

In [28]:
# Step 1: Drop rows where everything is NaN
# Create a list with the columns of interest
NaN_cols = ['start_station_name', 'end_station_name', 'end_lat']  

# Drop a row only if all of those columns are NaN
stations_df.dropna(subset=NaN_cols, how='all', inplace=True) 

stations_df.shape # to count how many rows were dropped

(29838166, 7)

no raws where dropped. that means that all missing values in end station has starting station, or coordinates. 

In [30]:
# Step 2a: Tagging Irrecoverable Data for start stations
# 1- Identify rows with no Name AND no GPS (creats Bolean_mask)
missing_start_mask = (
    stations_df['start_station_name'].isna()
    & (stations_df['start_lat'].isna() | stations_df['start_lng'].isna())
)
# 2- Tag them so they are included in counts but clearly labeled
stations_df.loc[missing_start_mask,'end_station_name'] = "Missing GPS and Name"

In [10]:
# repeat Step 2a: Tagging Irrecoverable Data for the missing start station coordinates
# 1 — Identify rows with missing start coordinates but available name
#missing_gps_mask1 = (
    stations_df['start_lat'].isna() |
    stations_df['start_lng'].isna()
)

# 2 — Tag the coordinate columns directly
#stations_df.loc[missing_gps_mask1, 'start_lat'] = "Missing GPS"
#stations_df.loc[missing_gps_mask1, 'start_lng'] = "Missing GPS"

In [31]:
# Step 2b: Tagging Irrecoverable Data for end station
# 1- Identify rows with no Name AND no GPS (creats Bolean_mask)
missing_end_mask = (
    stations_df['end_station_name'].isna()
    & (stations_df['end_lat'].isna() | stations_df['end_lng'].isna())
)
# 2- Tag them so they are included in counts but clearly labeled
stations_df.loc[missing_end_mask,'end_station_name'] = "Missing GPS and Name"

In [ ]:
# repeat Step 2b: Tagging Irrecoverable Data for the missing start station coordinates
# 1 — Identify rows with missing start coordinates but available name
#missing_gps_mask2 = (
    stations_df['end_lat'].isna() |
    stations_df['end_lng'].isna()
)

# 2 — Tag the coordinate columns directly
#stations_df.loc[missing_gps_mask2, 'end_lat'] = "Missing GPS"
#stations_df.loc[missing_gps_mask2, 'end_lng'] = "Missing GPS"

In [32]:
df_nan2= stations_df.isna().sum()
df_nan2

start_station_name       49
end_station_name      32661
start_lat                 0
start_lng                 0
end_lat               37223
end_lng               37223
value                     0
dtype: int64

In [34]:
stations_df.head()

,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,value
0,S 4 St & Wythe Ave,Kent Ave & S 11 St,40.712859,-73.965903,40.707645,-73.968415,1
1,Lexington Ave & E 120 St,Amsterdam Ave & W 125 St,40.801307,-73.939817,40.813358,-73.956461,1
2,Lexington Ave & E 120 St,Amsterdam Ave & W 125 St,40.801307,-73.939817,40.813358,-73.956461,1
3,Cleveland Pl & Spring St,Mott St & Prince St,40.722104,-73.997249,40.723180,-73.994800,1
4,Central Park North & Adam Clayton Powell Blvd,E 110 St & Madison Ave,40.799484,-73.955613,40.796154,-73.947821,1


#### Tagging metrics
69,884 - 32,661 = 37,223 end station name were taged as "Missing GPS and Name". exactly matches the missing number of coordinates. they have starting point but no end point. the starting stations could be identified but i am not doing this here.

32,661 end station name are missing but has coordinates. those can be recovered in the next steps.

37223 of missing end station coordinates will be drop out.

49 missing start station names have coordinates, therefore they remained untagged and the names can be restored from the coordiated later.

isna sum shows 0 NAN in the start coordinates i.e no standard missing data where found so far.

#### Next steps:
37,223 "Missing GPS and Name" rows will be droped from the df.



In [35]:
# Drop out the rows with missing information from the main dataset
stations_df = stations_df[
    (stations_df['end_station_name'] != "Missing GPS and Name") &
    (~stations_df['end_lat'].isna()) &
    (~stations_df['end_lng'].isna())
]

In [36]:
stations_df.shape

(29800943, 7)

In [43]:
df_nan3= stations_df.isna().sum()
df_nan3

start_station_name       49
end_station_name      32661
start_lat                 0
start_lng                 0
end_lat                   0
end_lng                   0
value                     0
dtype: int64

In [37]:
stations_df.dtypes

start_station_name     object
end_station_name       object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
value                   int64
dtype: object

In [38]:
# Check what values remain in the coordinates columns 
coord_cols = ['start_lat', 'start_lng', 'end_lat', 'end_lng']

# Find rows where ANY coordinate column is not numeric
non_numeric_mask = stations_df[coord_cols].apply(
    lambda col: pd.to_numeric(col, errors='coerce')
).isna().any(axis=1)

non_numeric_rows = stations_df[non_numeric_mask]

In [39]:
non_numeric_rows

,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,value


In [41]:
stations_df.head()

,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,value
0,S 4 St & Wythe Ave,Kent Ave & S 11 St,40.712859,-73.965903,40.707645,-73.968415,1
1,Lexington Ave & E 120 St,Amsterdam Ave & W 125 St,40.801307,-73.939817,40.813358,-73.956461,1
2,Lexington Ave & E 120 St,Amsterdam Ave & W 125 St,40.801307,-73.939817,40.813358,-73.956461,1
3,Cleveland Pl & Spring St,Mott St & Prince St,40.722104,-73.997249,40.723180,-73.994800,1
4,Central Park North & Adam Clayton Powell Blvd,E 110 St & Madison Ave,40.799484,-73.955613,40.796154,-73.947821,1


until here i have removed all rows with missing coordinates. and now the missing names will be restored

In [42]:
# Save the cleaned df 
# Get the full file path from .env
stations_df_path = os.getenv("STATIONS_DF")

# Read the CSV directly
stations_df.to_csv(stations_df_path)

## Restoring missing start station names


In [10]:
# Save the cleaned df 
# Get the full file path from .env
stations_df_path = os.getenv("STATIONS_DF")

# Read the CSV directly
stations_df=pd.read_csv(stations_df_path)

In [11]:
# Step 3A — Build a reference table (unique coordinates → station name)
ref_start_df = (
    stations_df
    .dropna(subset=['start_station_name', 'start_lat', 'start_lng'])
    .drop_duplicates(subset=['start_lat', 'start_lng'])
    [['start_lat', 'start_lng', 'start_station_name']]
)

In [12]:
# rename the headers to avoid replacing unmatched rows after merging
ref_start_df = ref_start_df.rename(columns={
    'start_station_name': 'start_station_name2',
    'start_lat': 'start_lat2',
    'start_lng': 'start_lng2'
})

In [13]:
ref_start_df.head()

,start_lat2,start_lng2,start_station_name2
0,40.712859,-73.965903,S 4 St & Wythe Ave
1,40.801307,-73.939817,Lexington Ave & E 120 St
3,40.722104,-73.997249,Cleveland Pl & Spring St
4,40.799484,-73.955613,Central Park North & Adam Clayton Powell Blvd
6,40.725029,-73.990697,E 2 St & 2 Ave


In [14]:
ref_start_df.shape


(4063778, 3)

In [15]:
stations_df = stations_df.merge(
    ref_start_df,
    left_on=['start_lat', 'start_lng'],
    right_on=['start_lat2', 'start_lng2'],
    how='left',
    indicator=True
)

In [16]:
stations_df['_merge'].value_counts()

_merge
both          29800894
left_only           49
right_only           0
Name: count, dtype: int64

49 "left_only" found in the _merge indicator column. That means, there was no merge. see below explanation

In [17]:
#Step 5A — Fill missing names using the reference values
stations_df['start_station_name'] = (
    stations_df['start_station_name']
    .fillna(stations_df['start_station_name2'])
)

In [18]:
df_nan3= stations_df.isna().sum()
df_nan3

Unnamed: 0                 0
start_station_name        49
end_station_name       32661
start_lat                  0
start_lng                  0
end_lat                    0
end_lng                    0
value                      0
start_lat2                49
start_lng2                49
start_station_name2       49
_merge                     0
dtype: int64

In [19]:
stations_df.shape

(29800943, 12)

In [20]:
stations_df = stations_df.drop(columns=['_merge'])

## Restoring missing end station names

In [ ]:
# Step 3B — Build a reference table (unique end coordinates → end station name)
ref_end_df = (
    stations_df
    .dropna(subset=['end_station_name', 'end_lat', 'end_lng'])
    .drop_duplicates(subset=['end_lat', 'end_lng'])
    [['end_lat', 'end_lng', 'end_station_name']]
)

In [ ]:
# rename the headers to avoid replacing unmatched raws after merging
ref_end_df = ref_end_df.rename(columns={
    'end_station_name': 'end_station_name2',
    'end_lat': 'end_lat2',
    'end_lng': 'end_lng2'
})

In [ ]:
ref_end_df.head()

,end_lat2,end_lng2,end_station_name2
0,40.707645,-73.968415,Kent Ave & S 11 St
1,40.813358,-73.956461,Amsterdam Ave & W 125 St
3,40.723180,-73.994800,Mott St & Prince St
4,40.796154,-73.947821,E 110 St & Madison Ave
6,40.720196,-73.989978,Allen St & Rivington St


In [ ]:
# Step 4B — Merge to attach recovered end station names
stations_df = stations_df.merge(
    ref_end_df,
    left_on=['end_lat', 'end_lng'],
    right_on=['end_lat2', 'end_lng2'],
    how='left',
    indicator=True
)

In [ ]:
# check up 1
stations_df['_merge'].value_counts()

_merge
both          29768282
left_only        32661
right_only           0
Name: count, dtype: int64

32661 left only means no merge . see explanation below

In [26]:
# check up 2
# Unique coordinates for missing end names:
missing_end_coords = (
    stations_df[stations_df['end_station_name'].isna()]
    [['end_lat', 'end_lng']]
    .drop_duplicates()
)


In [27]:
# check up 3
# Unique coordinates for known end names:
known_end_coords = (
    stations_df[stations_df['end_station_name'].notna()]
    [['end_lat', 'end_lng']]
    .drop_duplicates()
)

In [28]:
# Intersection
missing_end_coords.merge(
    known_end_coords,
    on=['end_lat', 'end_lng'],
    how='inner'
)

,end_lat,end_lng


#### Why the end-station merge failed?
the merge indicatore shows  23661 left only merge. that means all missing end station names failed to merge.
The merge for restoring end_station_name did not work because none of the coordinate pairs belonging to rows with missing end‑station names appear in any row that has a known end‑station name. In other words, the missing‑name coordinates and the known‑name coordinates form two completely separate sets with no overlap, so the lookup table had nothing to match against.
How I confirmed this
I ran a coordinate‑overlap test:
- Extracted unique coordinates for rows with missing end_station_name.
- Extracted unique coordinates for rows with known end_station_name.
- Performed an inner merge on end_lat + end_lng between the two sets.
The result contained 0 matching coordinate pairs, proving that the missing end‑station coordinates never occur alongside a known station name anywhere in the dataset. Therefore, the merge could not restore any of the missing names.

#### Why missing end station names cannot be restored from the existing coordinates?
This is actually very common in CitiBike trip data:
- Start stations are almost always known
- End stations are often missing when the bike is returned outside a dock
- e.g., a bike is left outside a station
- or GPS drift places it slightly off the official station coordinates
- or the system logs a “free‑floating” return
In those cases:
- The coordinates are real
- But they do not correspond to any official station
- So no lookup table can restore the name
This is why the start‑station restoration worked (because start stations are official),
but end‑station restoration failed (because many end points are not official stations).
#### Why 49 start station names could not be restored
the only technical explanation might be is that the coordinats in the merge is not accuratly mergen if for some reason they are changing. otherwise, it might be system registration failor.

## Final cleanup: Any remaining NaNs get a general "Unknown" label

In [29]:
# Label all missing NaN values in the start and end station names 
stations_df['start_station_name'] = stations_df['start_station_name'].fillna("Unknown Station")
stations_df['end_station_name'] = stations_df['end_station_name'].fillna("Unknown Station")

In [30]:
stations_df = stations_df.drop(columns=['_merge'])

In [31]:
df_nan4= stations_df.isna().sum()
df_nan4

Unnamed: 0                 0
start_station_name         0
end_station_name           0
start_lat                  0
start_lng                  0
end_lat                    0
end_lng                    0
value                      0
start_lat2                49
start_lng2                49
start_station_name2       49
end_lat2               32661
end_lng2               32661
end_station_name2      32661
dtype: int64

In [32]:
stations_df = stations_df.drop(columns=['start_lat2', 'start_lng2', 'end_lat2', 'end_lng2', 'start_station_name2', 'end_station_name2'])

In [33]:
stations_df.head()

,Unnamed: 0,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,value
0,0,S 4 St & Wythe Ave,Kent Ave & S 11 St,40.712859,-73.965903,40.707645,-73.968415,1
1,1,Lexington Ave & E 120 St,Amsterdam Ave & W 125 St,40.801307,-73.939817,40.813358,-73.956461,1
2,2,Lexington Ave & E 120 St,Amsterdam Ave & W 125 St,40.801307,-73.939817,40.813358,-73.956461,1
3,3,Cleveland Pl & Spring St,Mott St & Prince St,40.722104,-73.997249,40.723180,-73.994800,1
4,4,Central Park North & Adam Clayton Powell Blvd,E 110 St & Madison Ave,40.799484,-73.955613,40.796154,-73.947821,1


In [34]:
# Save the minimized df 
# Get the full file path from .env
stations2_df_path = os.getenv("STATIONS_REST_DF")

# Read the CSV directly
stations_df.to_csv(stations2_df_path)